In [1]:
from train_detect import *

In [2]:
train_cascade([1, 5, 10, 50])

Building features
Applying features to training examples
Len X10
Len Y10
Selected 63960 potential features
Chose classifier: <HaarLikeFeatures.WeakClassifier object at 0x000002632F1567F0> with accuracy: 10.000000 and alpha: inf
Stopping early. FPR = 0
False Positive Rate: 0/9 (0.000000)
False Negative Rate: 0/1 (0.000000)
Accuracy: 10/10 (1.000000)
Average Classification Time: 0.000600


c:\Users\LENOVO\Desktop\Img-project\Viola_Jones.py:68: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = math.log(1.0/beta)


In [4]:
test_cascade()

False Positive Rate: 1261/23573 (0.053493)
False Negative Rate: 431/472 (0.913136)
Accuracy: 22353/24045 (0.929632)
Average Classification Time: 0.000900
